In [ ]:
import sys
import subprocess

from pathlib import Path

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[1])

# Add to sys.path
if parent_dir not in sys.path:
  sys.path.insert(0, parent_dir)

from pprint import pprint
pprint(sys.path)

In [ ]:
import ffmpeg
import os

from pprint import pprint

### Basic Parameters

In [ ]:
vconf = {
  'crf': 18,
  'tune': 'stillimage',
  'vcodec': 'libx264',
  'vprofile': 'high',
}

aconf = {
  'acodec': 'aac',
}

oconf = {
  'strict': 'strict',
}

### ZoomPan

In [ ]:
from videogen.keyframe import keyframe

file = '/Volumes/RamDisk/1.mp4'

input = ffmpeg.input(file)
audio = input.audio
video = input.video

#  https://trac.ffmpeg.org/ticket/4298
v, name = keyframe(video)

stream = ffmpeg.output(
  v, audio, '/Volumes/RamDisk/out.mp4',
  **vconf, **aconf, **oconf,
).overwrite_output()

args = stream.compile()
print(name)
print(" ".join(args))

process = subprocess.Popen(
  args,
  stderr=subprocess.PIPE,
  stdout=subprocess.PIPE,
)
stdout, stderr = process.communicate()
if process.returncode != 0:
  pprint(args)
  print(stderr.decode('utf-8'))

In [ ]:
import IPython.display
from IPython.display import Video

IPython.display.display(Video('/Volumes/RamDisk/out.mp4'))

%%HTML
<video width="720" height="960" controls>
  <source src="/Volumes/RamDisk/out.mp4" type="video/mp4">
</video>

### Duration

In [ ]:
audio_file = ''
probe = ffmpeg.probe(audio_file)
audio = next((stream for stream in probe['streams'] if stream['codec_type'] == 'audio'), None)
durations = [float(stream['duration']) for stream in probe['streams'] if stream['duration'] is not None]
print(f'Duration: {audio["duration"]}')
max(durations) + 0.5

### Probe

In [ ]:
file = '/Volumes/RamDisk/temp/test_generate0/0.mp4'
ffmpeg.probe(file)

### Loop

In [ ]:
image = '1.jpg'
wav = '1.wav'
path = 'out.mp4'
width = 720
height = 960

video = ffmpeg.input(
  image,
  r=25,
  loop=1,
).video.filter(
  'scale',
  width=f'{width}',
  height='-2',
).filter(
  'pad',
  width=f'{width}',
  height=f'{height}',
  x='(ow-iw)/2',
  y='(oh-ih)/2',
  color='black',
).filter(
  'setsar',
  r='1',
  max='1'
)
stream = ffmpeg.output(
  video,
  'out.mp4',
  # r=25,  # set fps explicitly to support gif
  shortest=None,
  vcodec='libx264',
  acodec='aac',
  pix_fmt='yuvj420p',
  strict='experimental',
)

audio = ffmpeg.input(wav).audio.filter(
  # extend(padding) audio by 0.5s to make the concatenation more natural
  'apad',
  pad_dur=0.5,
).filter(
  'aformat',
  sample_fmts='fltp',
  sample_rates=44100,
)

stream = ffmpeg.concat(video, audio, v=1, a=1, n=2).output(
  path,
  # r=25,  # set fps explicitly to support gif
  shortest=None,
  vcodec='libx264',
  acodec='aac',
  pix_fmt='yuvj420p',
  strict='experimental',
)

stream = ffmpeg.output(video, audio, path,
  shortest=None,
  vcodec='libx264',
  acodec='aac',
  pix_fmt='yuvj420p',
  strict='experimental',
  overwrite_output=True,
)

stream = ffmpeg.filter(
  (audio, audio),
  'amerge',
  inputs=2,
  channel_layout='stereo',
).output(
  video,
  path,
  shortest=None,
  vcodec='libx264',
  acodec='aac',
  pix_fmt='yuvj420p',
  strict='experimental',
).overwrite_output()
        
args = stream.compile()
pprint(args)
print(' '.join(args))

In [ ]:
def test(**kwargs):
  return kwargs
test(**{'a':10, 'b':20}, **{'c':30})